# ML-6 Отбор и селекция признаков
###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Способы представления данных](#2)
- [3. Кодирование признаков](#3)
- [4. Обработка пропусков и выбросов](#4)
- [5. Масштабирование признаков](#5)
- [6. Трансформации распределений признаков](#6)
- [7. Даты и расстояния](#7)
- [8. Отбор признаков: Мотивация](#8)
- [9. Отбор признаков: Классификация методов](#9)
- [10. Практика](#10)
- [11. Итоги ](#11)

# 2. Способы представления данных <a class="anchor" id=2></a>

1. Числа

2. Не числа (их нахуй)

[к содержанию](#0)

# 3. Кодирование признаков <a class="anchor" id=3></a>

## Можно посмотреть файл `codding.ipynb`

[к содержанию](#0)

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [3]:
data = pd.read_excel('data_ford_price.xlsx') 
y = data['price']
x = data.drop(columns='price')

Попробуем применить линейную регрессию на «сырых» данных:

In [4]:
lr = LinearRegression()
lr.fit(x,y)

ValueError: could not convert string to float: 'clean'

Мы получим ошибку с комментарием о том, что не удалось превратить строковое значение в число с плавающей точкой (`float`).

Чтобы этой ошибки не возникало, необходимо **закодировать данные.**

<img src=ml6_img1.png>

В таблице ниже представлено сравнение данных способов кодировки:

<img src=ml6_img2.png>

Для реализации данных методов мы использовали библиотеку [category_encoders](https://contrib.scikit-learn.org/category_encoders/). Однако вы уже знаете и такой мощный инструмент, как [scikit-learn](https://scikit-learn.org/). Данная библиотека содержит набор реализованных алгоритмов машинного обучения, метрик для оценки их качества, а также  класс preprocessing для предобработки данных, в частности — для кодирования категориальных признаков.

Представленная ниже таблица показывает соответствие типа кодирования классу в `sklearn.processing`.

<img src=ml6_img3.png>

Преимущество использования одной библиотеки состоит в типичности методов. Например, вы знаете, что для обучения модели в sklearn используется метод `fit()`. При кодировании признаков здесь также применяют `fit()` для подгонки кодировщика под выборку и `transform()` — для преобразования данных в числа.

<img src=ml6_img4.png>

Из предыдущих модулей мы знаем, что при решении задач машинного обучения данные разбираются на обучающую (`train`) и валидационную (`validation`) выборки (последняя также может быть тестовой (`test`) выборкой). По аналогии подгонка кодировщика происходит на обучающей выборке, а трансформация — на обучающей и на тестовой.

Почему так? Потому что наша обученная модель не должна видеть данные, которые подаются в неё на тесте. Только так мы можем судить о том, что модель обучена качественно. То же самое и с кодировкой.

Давайте посмотрим на кодирование признака Образование способом **«один-против-всех»** (`one vs all`):

In [5]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
 
lb.fit(education)
 
print('категории:', lb.classes_)
 
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

У класса `LabelBinarizer`, как и у двух остальных, есть атрибут `classes_`, который выводит список уникальных значений признака.

Вернёмся к нашей выборке. В ней присутствуют следующие категориальные признаки: `condition`, `cylinders`, `title_status`, `transmission`, `drive`, `size`.

При этом, признаки: 
* `condition` и `cylinders` — **числовые**

*  `title_status`, `transmission`, `drive`, `size` —**текстовые**.

<img src=ml6_img5.png>

### Важно производить кодирование номинальных признаков, даже если они уже представлены в числовом формате, так как, в отличие от порядковых признаков, **категории номинальных являются независимыми**. В случае порядкового кодирования таких признаков мы вносим искусственные закономерности в данные (например, чем больше числовой код цилиндров, тем лучше, хотя это необязательно так).

Посмотрим на число уникальных значений номинальных признаков `title_status`, `transmission`, `drive`, `size` и `cylinders`:

In [6]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений признака {}: '.format(column), data[column].nunique())

Число уникальных значений признака cylinders:  6
Число уникальных значений признака title_status:  5
Число уникальных значений признака transmission:  3
Число уникальных значений признака drive:  3
Число уникальных значений признака size:  4


Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — `fit_transform()`. В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода `get_feature_names()` получим список новых названий колонок:

In [8]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names(columns_to_change)
print(column_names)

AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names'

In [12]:
data_new = pd.get_dummies(data)
data_new.shape

(7017, 23)

# 4. Обработка пропусков и выбросов <a class="anchor" id=4></a>

[к содержанию](#0)

## Можно посмотреть `NaNs.ipynb`



In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [14]:
data = pd.read_excel('data_ford_price.xlsx') 
data.isna().sum()

price              0
year               0
condition          0
cylinders          0
odometer           0
title_status       0
transmission       0
drive            391
size            1564
lat                0
long               0
weather          180
dtype: int64

# 5. Масштабирование признаков <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Трансформации распределений признаков <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Даты и расстояния <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Отбор признаков: Мотивация <a class="anchor" id=8></a>

[к содержанию](#0)

# 9. Отбор признаков: Классификация методов <a class="anchor" id=9></a>

[к содержанию](#0)

# 10. Практика <a class="anchor" id=10></a>

[к содержанию](#0)

# 11. Итоги <a class="anchor" id=11></a>

[к содержанию](#0)